In [1]:
import numpy as np
import pandas as pd
import random
from typing import Dict,Union,Any
from collections import defaultdict

In [2]:
A = np.random.uniform((10, 25),size = (2,2))
B = np.random.uniform((10, 25),size = (1,2))
C = np.vstack((A,B))
C

array([[ 1.41815457, 16.41833191],
       [ 2.97254693, 16.66495772],
       [ 9.23743458, 20.70592685]])

In [3]:
np.sum(C)

np.float64(67.41735254766553)

In [4]:
# Constants
random.seed(1000)
np.random.seed(1000)
n_inbound = 30
n_platoon = 4
min_railcar = 3
sizes = [1.2,3.6,6.0,12.0]
truck_len = 16.0
n_dest = 5
changeover_time = 30
n_stack = 4

# Arrival and Deadline generation
Arrival_platoon = np.sort(np.round(np.random.uniform(150, 600, n_platoon)))
Arrival_platoon = np.round([(max(Arrival_platoon[i-1]+50,Arrival_platoon[i])) if i!= 0 else Arrival_platoon[i] for i in range(len(Arrival_platoon))])
#
Dest_dict : Dict[int,list[int]] = {i:[] for i in range(n_dest)}

# Penalty
#Penalty = np.round(np.full(n_outbound, 3))

Breakpt = [120, 240]
Multiplier = [1, 2, 4]


def random_sup_dem_gen(n_inbound, n_size, n_dest, Dest_dict):
    # random.seed(1000)
    # np.random.seed(1000)
    
    Supply = np.zeros((n_inbound, n_dest, n_size), dtype=int)
    Truck_dest = []
    
    size_selection = {el: 1 for el in sizes}
    dest_selection = {el: 1 for el in range(n_dest)}
    size_weights = np.array([1/(size_selection[el])**0.5 for el in sizes])
    dest_weights = np.array([1/(dest_selection[el])**0.5 for el in range(n_dest)])
    norm_size = [i/sum(size_weights) for i in size_weights]
    norm_dest = [i/sum(dest_weights) for i in dest_weights]
    for i in range(n_inbound):
        rem_len = truck_len
        size_list = sizes
        dum_weights = [size_weights[i] for i,size in enumerate(sizes) if size in size_list]
        dum_norm = [i/sum(dum_weights) for i in dum_weights]
        while rem_len > 0:
            size_sel = np.random.choice(size_list,p=dum_norm)
            size_selection[size_sel] += 1;
            size_weights[sizes.index(size_sel)] = 1/(size_selection[size_sel])**0.5
            dest_sel = np.random.choice(list(range(n_dest)),p=norm_dest)
            dest_selection[dest_sel] += 1;
            dest_weights[dest_sel] = 1/(dest_selection[dest_sel])**0.5
            Supply[i,dest_sel,sizes.index(size_sel)] += 1
            rem_len -= size_sel
            size_list = [x for x in sizes if x <= rem_len]
            if len(size_list) <= 0:
                break;
            else:
                dum_weights = [size_weights[i] for i,size in enumerate(sizes) if size in size_list];
                dum_norm = [i/sum(dum_weights) for i in dum_weights];
                norm_dest = [i/sum(dest_weights) for i in dest_weights];

    #print(Supply)

    Demand = []
    rem_supply = np.zeros((n_dest,n_size), dtype=int)
    dest_selection = {el: 1 for el in range(n_dest)}
    dest_weights = np.array([1/(dest_selection[el])**0.5 for el in range(n_dest)])
    norm_dest = [i/sum(dest_weights) for i in dest_weights]
    for i in range(n_dest):
        for j in range(n_size):
            rem_supply[i,j] = sum(Supply[:,i,j])


    truck_cnt = 1

    while np.sum(rem_supply) > 0:
        place_demand = [0]*n_size
        rem_len = truck_len
        sel_dest = np.random.choice(list(range(n_dest)),p=norm_dest)
        # print(f'Initial Dest - {sel_dest} and the supply is {rem_supply[:,sel_dest]}')
        if sum(rem_supply[sel_dest,:]) == 0:
            while True:
                sel_dest = np.random.choice(list(range(n_dest)),p=norm_dest)
                if sum(rem_supply[sel_dest,:]) > 0:
                    # print(f'Change Dest - {sel_dest} and the supply is {rem_supply[:,sel_dest]}')
                    break;
        dest_selection[sel_dest] += 1;
        dest_weights[sel_dest] = 1/(dest_selection[sel_dest])**0.5
        norm_dest = [i/sum(dest_weights) for i in dest_weights]
        Dest_dict[sel_dest].append(truck_cnt)
        
        Truck_dest.append(sel_dest)

        # get various list of sizes for a destination
        Supply_alloc = [i for i in range(n_size) if rem_supply[sel_dest,i] > 0]
        #print(Supply_alloc)
        # break;
        while rem_len > 0:
            # print(f'Sizes available {Supply_alloc}')
            # print(f'Length available {rem_len}')
            size_ind = random.choice(Supply_alloc)
            #print(size_ind)
            # break;
            place_demand[size_ind] += 1
            rem_supply[sel_dest,size_ind] -= 1
            rem_len -= sizes[size_ind]
            Supply_alloc = [i for i,size in enumerate(sizes) if size <= rem_len and rem_supply[sel_dest,i]>0]
            if len(Supply_alloc) <= 0:
                break;
        Demand.append(place_demand)
        truck_cnt += 1;
        

    Demand = np.array(Demand)
 
    return Supply, Demand, Dest_dict, np.array(Truck_dest)

# Run generation
Supply, Demand, Dest_dict, Truck_dest = random_sup_dem_gen(n_inbound, len(sizes), n_dest, Dest_dict)
print(f'Total number of containers supplied by railcars is {np.sum(Supply)}')
print(f'Total number of containers taken by {len(Demand[:,0])} trucks is {np.sum(Demand)}')
n_outbound = len(Demand[:,0])

Arrival_trucks = np.round(np.random.uniform(150, 600, n_outbound))

Deadline_trucks = np.round(np.random.uniform(350, 1000, n_outbound))

Deadline_trucks = np.round([(max(Arrival_trucks[i]+150,Deadline_trucks[i])) for i in range(len(Deadline_trucks))])


for i in sorted(Dest_dict.keys()):
    print(f'Destination{i} trucks are {Dest_dict[i]}')


# Demand max per outbound
# Demand_max = Demand.sum(axis=1)

# # # Write to CSV
# filename = f"Supply_Demand_exact_{n_inbound}_{n_outbound}.csv"
# with open(filename, "w") as f:
#     headers = ["Rail_num", "Car_num", "Arrival_time", "Deadline", "Supply/Demand/Loadtime"] + [f"P{p+1}" for p in range(len(sizes))]
#     f.write(",".join(headers) + "\n")

#     load_row = ["NA", "NA", "min", "min", "Loadtime"] + list(map(str, LoadTime))
#     f.write(",".join(load_row) + "\n")

    # for platoon in pla

#     for i in range(n_inbound):
#         row = [f"IT{i+1}", "Inbound trailer", str(int(Arrival[i])), "NA", "Supply"] + list(map(str, Supply[i]))
#         f.write(",".join(row) + "\n")

#     for j in range(n_outbound):
#         row = [f"OT{j+1}", "Outbound trailer", "NA", str(int(DeadLines[j])), "Demand"] + list(map(str, Demand[j]))
#         f.write(",".join(row) + "\n")

Total number of containers supplied by railcars is 117
Total number of containers taken by 34 trucks is 117
Destination0 trucks are [3, 4, 9, 12, 18, 27, 30, 32, 34]
Destination1 trucks are [2, 5, 10, 13, 16, 22, 24, 31, 33]
Destination2 trucks are [7, 19, 23, 26, 29]
Destination3 trucks are [1, 6, 8, 15, 20]
Destination4 trucks are [11, 14, 17, 21, 25, 28]


In [5]:
random.seed(1000)
np.random.seed(1000)

class Platoon:
    capacity = n_stack;
    def __init__(self,rail_num,timestamp):
        self.rail_num = rail_num
        self.list = [];
        self.front = 0;  # front railcar at the dock 
        self.rear = 0;   # last railcar at the dock
        self.direct = 0;  # proxy for the word pass
        self.size = 0;
        self.front_right = Platoon.capacity;
        self.timestamp = timestamp
        self.history = [timestamp]


    def _init_enqueue(self,item):

        self.list.append(item)
        self.size += 1;
        
        if self.size <= Platoon.capacity:
            if self.size > 1:
                self.rear += 1

    def update_timestamp(self,time_val):

        if self.timestamp > time_val:
            raise ValueError(f"Timestamp should be greater than {time_val}")
        else:
            self.timestamp = time_val
            self.history.append(self.timestamp)

    def remain(self,Sup_req):
        return sum([np.sum(Sup_req[railcar,:,:]) for railcar in self.list])

    def remain_cont(self,Sup_req):
        return [np.sum(Sup_req[railcar,:,:]) for railcar in self.list]
        

    def len_platoon(self):
        return self.size

    def slot_position(self,item):

        try:
            sel_ind = self.list.index(item)
            if sel_ind < self.front:
                print(f"The following railcar {item} lies outside the dock to the left")
            elif sel_ind > self.rear:
                print(f"The following railcar {item} lies outside the dock to the right")
            else:
                return sel_ind - self.front + 1 # to return the docking position of the truck
        except ValueError:
            print(f"The given railcar {item} is not present in platoon number {self.rail_num}")
            

    def dock_queue(self):
        if self.direct % 2 == 0:
            return self.list[self.front:self.rear+1]
        else:
            return list(reversed(self.list[self.front:self.rear+1]))
        
        
    def right_queue(self):

        if self.rear >= (self.size-1):
            print("The queue on the right doesn't exist")
            return []
        else:
            return self.list[self.rear+1:]

    def left_queue(self):

        if self.front == 0:
            print("The queue on the left doesn't exist")
            return []
        else:
            return self.list[:self.front]

    def shift_platoon(self):

        if self.direct % 2 == 0:
            self.shift_bogie_left()
        else:
            self.shift_bogie_right()

        return;

    def shift_secondary(self,Supply_req,time_val):

        time_flag = 0;
        if sum([np.sum(Supply_req[car,:,:]) for car in self.list]) > 0:
        
            if self.len_platoon() > n_stack:
                # add a while loop until sum of products inside the dock is not zero
                move_flag = 0;
                railcar_dock = self.dock_queue()
                int_len = len(railcar_dock)
                prev_direct = platoon.direct;
                if prev_direct % 2 == 0:
                    if sum([np.sum(Supply_req[car,:,:]) for car in platoon.right_queue()]) == 0:
                        platoon.direct += 1;
                else:
                    if sum([np.sum(Supply_req[car,:,:]) for car in platoon.left_queue()]) == 0:
                        platoon.direct += 1;

                railcar_dock = self.dock_queue()
                # if platoon.direct % 2 != 0:
                #     railcar_dock = list(reversed(railcar_dock))
                # i got it - ddnrekjefch
                # the list is always explored from left to right, the direction changes has to be explored from right to left
                for railcar in railcar_dock:
                    if np.sum(Supply_req[railcar,:,:]) <= 0:
                        if move_flag == 0:
                            prev_direct = self.direct;
                            if prev_direct % 2 == 0:
                                if sum([np.sum(Supply_req[car,:,:]) for car in self.right_queue()]) == 0:
                                    self.direct += 1;
                                    break;
                            else:
                                if sum([np.sum(Supply_req[car,:,:]) for car in self.left_queue()]) == 0:
                                    self.direct += 1;
                                    break;
                            self.shift_platoon()
                            time_flag = +1;
                    else:
                        move_flag = 1;
                        break;
                    if prev_direct != self.direct:
                        break;
                        
                # if all cars in the cross_dock are shifted and the coming cars are all empty
                # you arrive at this block only when the cars out of the dock still have containers
                
                if time_flag == int_len:
                    prev_direct = self.direct
                    if self.direct % 2 == 0:
                        while np.sum(Supply_req[self.dock_queue()[0],:,:]) == 0:
                    # if all the previous cars are empty; then shift another non-empty n_stack rail cars into the dock
                            self.shift_platoon()
                            if prev_direct != self.direct:
                                break;
                    else:
                        while np.sum(Supply_req[self.dock_queue()[-1],:,:]) == 0:
                            self.shift_platoon()
                            if prev_direct != self.direct:
                                break;
    
                if time_flag > 0: 
                    platoon.update_timestamp(time_val)
                    
        return;
        
            
    # both shift functions should be called only when the platoon is longer than the cross-dock
    def shift_bogie_left(self):
        

        if len(self.list) <= Platoon.capacity:
            print("Not required to call function for this platoon")
            return
        
        right_length = len(self.right_queue())
        
        if right_length > 0:
            self.front += 1;
            self.rear += 1;
            #self.front_right += 1
            if self.rear == len(self.list)-1:
                self.direct += 1; #( enough to increase pass only when self.rear reaches the last element for the first time) 
                                    # if the entire platoon is inside the dock,pass becomes unnecessary.)
        return;
            
            

    def shift_bogie_right(self):
        
        if len(self.list) <= Platoon.capacity:
            print("Not required to call function for this platoon")
            return
            
        
        left_length = len(self.left_queue())
        
        if left_length > 0:
            self.front -= 1;
            self.rear -= 1;
            #self.front_right -= 1;
            if self.front == 0: 
                self.direct += 1;

        return;
           
def distribute_items(total_items, bins, min_per_bin):
    # Step 1: Give each bin the minimum required
    distribution = [min_per_bin] * bins
    remaining = total_items - (min_per_bin * bins)

    # Step 2: Distribute remaining items randomly
    for _ in range(remaining):
        bin_index = random.randint(0, bins - 1)
        distribution[bin_index] += 1

    return distribution

platoon_num = distribute_items(n_inbound,n_platoon,min_railcar)
# print(f'Resultant distribution is {platoon_num}')
# print(f'Arrival times for platoons is {Arrival_platoon}')
# print(f'Arrival times for trucks is {Arrival_trucks}')
# print(f'Deadline times for trucks is {Deadline_trucks}')

def transfer_tool(arr1,arr2): # first array( demand by truck) , second array ( supply by storage or railcar)
    # both arrays are single dimensional arrays (only exchange products of various sizes)
    common_array = np.array([min(arr1[p],arr2[p]) for p in range(len(arr1))])
    arr1 = arr1 - common_array
    arr1[arr1 < 0] = 0
    arr2 = arr2 - common_array
    arr2[arr2 < 0] = 0

    return arr1,arr2,common_array

In [6]:
import numpy as np
from typing import Dict,Union,Any
# the key in inventory might be unnecessary
# use inventory as follows

inventory : Dict[int,Dict[int,Dict[str,Union[np.ndarray,int]]]] = {}
Truck_space : Dict[int,Dict[str,Union[np.ndarray,int]]] = {}
# the array type is going to be of the form dest*size ( a 2-d array)
# the truck space below wille be better suited for creating a list
Truck_space[1] = {"storage":np.array([1,2,3,4,5]),"timestamp":120}
inventory[0] = Truck_space
0 in inventory.keys()

True

In [7]:
# quantities measured in modular containers rather than just units 

class Storage:
    # possibly to tally occupied space inside storage
    def __init__(self, num_lanes, width):
        self.num_lanes = num_lanes
        self.width = width
        self.total_capacity = num_lanes*width #(length of each lane*number of lanes)
        self.used_capacity = 0 # measured in space occupied (m^2 or m length) pass - railcar - 
        self.inventory = np.zeros((num_lanes,n_dest,len(sizes)), dtype = int)
        self.space = np.array([width]*num_lanes)
        #self.inventory : Dict[int,Dict[int,Dict[str,Union[np.ndarray,int]]]] = {}
        self.timestamp = 0
        self.history = [] # needed: at each time keep track of when products are available,

    
    def update_timestamp(self,time_val):

        if self.timestamp > time_val:
            raise ValueError(f"Timestamp for storage should be greater than {time_val}")
        else:
            self.timestamp = time_val
            self.history.append(self.timestamp)
    
    
    
    def bin_packing(self,platoon,Supply_req,time_req):

        # trans_cnt if products moved from railcars to storage.
        # c for unloading, another c for transporting to storage
        trans_cnt = 0;
        rail_line = platoon.dock_queue()
        if sum([np.sum(Supply_req[car,:,:]) for car in rail_line]) > 0:
            for car in rail_line:
                for i in range(len(Supply_req[car,:,0])): # destination 
                    for j in range(len(Supply_req[car,0,:])): # size
                        cnt = 0;
                        for item in range(Supply_req[car,i,j]):
                            for k in range(len(self.space)):
                                if self.space[k] >= sizes[j]:
                                    self.space[k] -= sizes[j]
                                    self.inventory[k,i,j] += 1;
                                    cnt += 1;
                                    trans_cnt = 1;
                                    break; # to move on to the next item
                                    
                        Supply_req[car,i,j] -= cnt;
            if trans_cnt == 0:
                return Supply_req,trans_cnt;

            # timestamp after only unloading
            platoon.update_timestamp(time_req+c*trans_cnt)
            # timestamp after trnasferring to the storage as well
            if trans_cnt == 1:
                self.update_timestamp(time_req+2*c*trans_cnt)
            platoon.shift_secondary(Supply_req,time_req+c*trans_cnt)

        else:
            platoon.update_timestamp(time_req)
            platoon.shift_secondary(Supply_req,time_req)
        
        return Supply_req,trans_cnt;


    def storage_to_truck(self,Demand_req,truck):

        req_dest = Truck_dest[truck]
        stor_transfer = np.zeros(len(Demand_req[truck,:]),dtype = int);

        for k in range(len(self.inventory)): # k is the lane number
            if sum(self.inventory[k,req_dest,:]) > 0:
                Demand_req[truck,:],self.inventory[k,req_dest,:],transfer_amount = transfer_tool(Demand_req[truck,:],self.inventory[k,req_dest,:])
                stor_transfer += transfer_amount;
                for i in range(len(transfer_amount)):
                    self.space[k] += transfer_amount[i]*sizes[i]

        return Demand_req,stor_transfer

    def explore_storage_to_truck(self,Demand_req,truck):

        req_dest = Truck_dest[truck]
        stor_transfer = np.zeros(len(Demand_req[truck,:]),dtype = int);
        Dummy_demand = Demand_req[truck,:].copy()
        Dummy_inventory = self.inventory.copy()

        for k in range(len(Dummy_inventory)): # k is the lane number
            if sum(Dummy_inventory[k,req_dest,:]) > 0:
                Dummy_demand,Dummy_inventory[k,req_dest,:],transfer_amount = transfer_tool(Dummy_demand,Dummy_inventory[k,req_dest,:])
                # stor_transfer += transfer_amount;
                # for i in range(len(transfer_amount)):
                #     self.space[k] += transfer_amount[i]*sizes[i]

        return Dummy_demand,stor_transfer

    def second_bin_packing(self,platoon,Supply_req,time_req):

        trans_cnt = 0;
        
        rail_line = platoon.dock_queue()
        for car in rail_line:
            for i in range(len(Supply_req[car,:,0])): # destination 
                for j in range(len(Supply_req[car,0,:])): # size
                    cnt = 0;
                    for item in range(Supply_req[car,i,j]):
                        for k in range(len(self.space)):
                            if self.space[k] >= sizes[j]:
                                self.space[k] -= sizes[j]
                                self.inventory[k,i,j] += 1;
                                cnt += 1;
                                trans_cnt = 1;
                                break; # to move on to the next item
                                
                    Supply_req[car,i,j] -= cnt;
                    
        if trans_cnt == 1:
            self.update_timestamp(time_req+2*c*trans_cnt)
        platoon.update_timestamp(time_req+c*trans_cnt)

        #-----------------the next loop is only concerned with shifting cars, not related to storage-----------------------#
        #------------------Start the loop for shifting trucks-------------------------#

        if platoon.remain(Supply_req) > 0:
            if platoon.len_platoon() > n_stack:
                # add a while loop until sum of products inside the dock is not zero
                #move_flag = 0;
                
                railcar_dock = platoon.dock_queue()
                int_len = len(railcar_dock)
                # this block to change direction initially
                prev_direct = platoon.direct;
                if prev_direct % 2 == 0:
                    if sum([np.sum(Supply_req[car,:,:]) for car in platoon.right_queue()]) == 0:
                        platoon.direct += 1;
                else:
                    if sum([np.sum(Supply_req[car,:,:]) for car in platoon.left_queue()]) == 0:
                        platoon.direct += 1;

                
                for item in railcar_dock:
                    prev_direct = platoon.direct
                    platoon.shift_platoon()
                    if prev_direct != platoon.direct :
                        break;

                if sum([np.sum(Supply_req[car,:,:]) for car in platoon.dock_queue()]) == 0:
                    prev_direct = platoon.direct
                    if platoon.direct % 2 == 0:
                        while np.sum(Supply_req[platoon.dock_queue()[0],:,:]) == 0:
                    # if all the previous cars are empty; then shift another non-empty n_stack rail cars into the dock
                            platoon.shift_platoon()
                            if prev_direct != platoon.direct:
                                break;
                    else:
                        while np.sum(Supply_req[platoon.dock_queue()[-1],:,:]) == 0:
                            platoon.shift_platoon()
                            if prev_direct != platoon.direct:
                                break;

        return Supply_req;
                
        

In [8]:
a = [1,2,3,4,5] 
b = list(reversed(a))
b

[5, 4, 3, 2, 1]

In [9]:
platoon_list = []
rail_cnt = 0;
for i in range(n_platoon):
    new_platoon = Platoon(i,Arrival_platoon[i])
    for j in range(platoon_num[i]):
        new_platoon._init_enqueue(rail_cnt)
        rail_cnt += 1;
    platoon_list.append(new_platoon)
    
Earl_Avail_Stack = np.zeros(n_stack, dtype=float)
#Earl_Dock_Stack = np.zeros(n_stack, dtype=float)
Entry_time = np.zeros(n_outbound, dtype=float)
Leave_time = np.zeros(n_outbound, dtype=float)
Seq_OT = [i for i in range(n_outbound)]
random.shuffle(Seq_OT)
preempt_list = []
n_preempt = np.zeros(n_outbound, dtype=int) # update when pushing the truck into a pre-empt list
Entry_platoon = {}
Leave_platoon = {}
Demand_iter = Demand.copy()
Supply_iter = Supply.copy()
#Load_Trans = np.zeros((n_inbound,n_outbound,n_products), dtype = int)
Zone = Storage(n_stack,2*truck_len)
# to track which doors are presently occupied
Free_doors = np.ones(n_stack, dtype = bool)
# to track which door each outbound truck is present at
Stack_Door = np.clip(np.zeros(n_stack, dtype = int),0,n_outbound)
Inv_Stack_Door = np.clip(np.zeros(n_outbound, dtype = int),1,n_stack)
# Seq_OT = sequence of outbound trucks
# Arrival[platoon]
OT_Leave : Dict[int,list[int]] = {i:[] for i in range(n_outbound)}

# initialize trucks present in the cross-dock
OT_dock = [];        

def sort_order(Enter_Dock,Truck_list):

    #sort_dock = sorted(Earl_Dock)
    req_dict = {}
    for i in Truck_list:
        req_dict[i] = Enter_Dock[i]
        
    sort_list = [k for k, v in sorted(req_dict.items(), key=lambda item: item[1])]
   
    return sort_list

def insert_truck(dock_list,rem_truck,req_truck,timestamp,time_list):
    dock_list.remove(rem_truck)
    for i,truck in enumerate(dock_list):
        if timestamp < time_list[truck]:
            dock_list.insert(i,req_truck)
            return dock_list

    return dock_list+[req_truck]
        

# def transport(platoon,railcar_dock,OT_dock,Seq_OT)


# cnt used to tally the number of outbound trucks in the sequence
cnt = 0;
for door in range(n_stack):
    sel_truck = Seq_OT[cnt]
    sel_door = np.argmin(Earl_Avail_Stack)
    Inv_Stack_Door[sel_truck] = sel_door+1
    Stack_Door[sel_door] = sel_truck
    req_time = max(Arrival_trucks[sel_truck],Earl_Avail_Stack[sel_door])
    Entry_time[sel_truck] = req_time
    #Earl_Dock_Stack[sel_door] = max(Arrival[sel_truck],Earl_Avail_Stack[sel_door])
    OT_dock.append(sel_truck)
    if cnt == len(Seq_OT)-1:
        break;
    cnt += 1;

c = 15
wait_time = 2*c

#prev_platoon_dep = 0;

trans_carry = 0;


for platoon in platoon_list[:3]:
    #platoon_ind += 1
    prev_platoon_dep = 0;
   
    platoon_duration = 0;
    OT_dock = sort_order(Entry_time,OT_dock)
    railcar_dock = platoon.dock_queue()
    platoon_ind = platoon_list.index(platoon)
    if platoon_ind != 0:
        Entry_platoon[platoon_ind] = max(Arrival_platoon[platoon_ind],Leave_platoon[platoon_ind-1]+c)
    else:
        Entry_platoon[platoon_ind] = Arrival_platoon[platoon_ind]
    platoon.update_timestamp(Entry_platoon[platoon_ind])
    # at beginning of each each pass list the outbound trucks that remain at the cross-dock
    # pass_comp : Dict[int,list[int]] = {}
    # pass_comp[platoon.direct] = OT_dock
    prev_platoon_stor = 0;
    platoon_stor = 0;
    platoon_while = 0; 
    # Leave_platoon[platoon] = max(various assortment of leave times) helpful to track leave time of
    # outer-pre-emption rely on wether the supply on the entire railcar remains the same between two iterations
    # Iter = 1;
    print(f"-------------------------------------------For platoon number {platoon_ind} whose arrival is {Entry_platoon[platoon_ind]}---------------------------------------------------------------")
    while platoon.remain(Supply_iter) > 0: # until supply left in the railcars is non-zero
        # after running the secondary while loop_ need to refresh OT_dock
        # OT_dock = sort_order(Entry_time,OT_dock
        
        prev_platoon_stor = platoon.remain(Supply_iter)

        railcar_dock = platoon.dock_queue()
        index = 0;
        # introdue a function
        # railcar_dock,OT_dock,move_flag = transport(platoon,railcar_dock,OT_dock,Seq_OT)
        place_hold = 0;
        while index < len(OT_dock):
            truck = OT_dock[index]
            req_dest = Truck_dest[truck]
            print(f"----------Index number {index} for truck {truck} whose arrival is {Entry_time[truck]}--------------------")
            print(f"The required set of trucks {OT_dock}")
            if platoon.remain(Supply_iter) > 0:
                while Entry_time[truck] > platoon.timestamp + wait_time: # (note make sure that containers are stored before truck enters; wait_time = c)
                    print("Check at while loop 1")
                    print(f"Entry time of truck {truck} is {Entry_time[truck]}")
                    print(f"At timestamp {platoon.timestamp} the trailers are {platoon.dock_queue()} with the containers being {[np.sum(Supply_iter[railcar,:,:]) for railcar in platoon.dock_queue()]}")
                    # this was supposed to be for a dock queue which should have some containers
                    Supply_iter,break_signal = Zone.bin_packing(platoon,Supply_iter,platoon.timestamp)
                    if break_signal != 0:
                        print(f"After transferring to storage, at timestamp {platoon.timestamp} the trailers are {platoon.dock_queue()} with the containers being {[np.sum(Supply_iter[railcar,:,:]) for railcar in platoon.dock_queue()]}")
                    # when shifting, update timestamp when products flow from cars to storage
                    if platoon.remain(Supply_iter) == 0:
                        #print(f"Important: Platoon emptied out here")
                        if prev_platoon_dep == 0:
                            prev_platoon_dep = platoon.timestamp
                            Leave_platoon[platoon_ind] = prev_platoon_dep
                        break;
                    if break_signal == 0:
                        break;

            Demand_iter,stor_transfer = Zone.storage_to_truck(Demand_iter,truck)
            if sum(stor_transfer) > 0:
                # on using the expression below possible corruption when carried over to the next platoon
                # need timestamp for storage when the first truck requires loop 2 buy the second truck only needs this loop, screwing up timestamp for stored goods
                # the idea is to update timestamp for storage when something is added and taken. 
                print("There is transfer from storage")
                # the time underneath will definetely depend on timestamp at the storage zone
                OT_Leave[truck].append(max(Entry_time[truck],Zone.timestamp)+2*c)
                #stor_next_truck = 0;
                if index < len(OT_dock)-1:
                    stor_next_truck = OT_dock[index+1]
                # else:
                #     stor_next_truck = OT_dock[0]
                    Dummy_trans,dummy_stor_transfer = Zone.explore_storage_to_truck(Demand_iter,stor_next_truck)
                    if sum(dummy_stor_transfer) != 0:
                        if sum(Dummy_trans) != 0:
                            # timestamp after completed transfer to the truck from the storage
                            # if entire next truck is filled out then only the timestamp remains the same, otherwise the timestamp changes,
                            # as the next truck has to wait for the next batch of containers into storage
                            Zone.update_timestamp(OT_Leave[truck][-1])
                        # else, the zone timestamp should remain the same
                    
                # only update timestamp when the next truck requires containers other than from storage.
                # an idea use platoon.timestamp when this truck in the set gets carried over to the next iteration of platoon
                
            # after clearing out, if none are 
            
            railcar_dock = platoon.dock_queue()
            # if there is any direct transfer between the railcars and trucks
            trans_flag = 0;

            
            if sum([np.sum(Supply_iter[railcar,:,:]) for railcar in railcar_dock]) > 0:
                # since containers are still present on the platoon, timestamp can still be updated.
                round_flag = 0;
                while sum(Demand_iter[truck,:]) > 0:
                    print("Check at while loop 2")
                    move_flag = 0;
                    travel_flag = 0;
                    direct_flag = 0;
                    railcar_dock = platoon.dock_queue()
                    rail_truck_array = np.zeros(len(sizes),dtype = int)

                    for railcar in railcar_dock:
                        Demand_iter[truck,:],Supply_iter[railcar,req_dest,:],trans_array = transfer_tool(Demand_iter[truck,:],Supply_iter[railcar,req_dest,:])
                        rail_truck_array += trans_array
                        # if there was a change in direction of the platoon, avoid the block that moves the platoon around 
                        if direct_flag == 1:
                            continue;
                        if np.sum(Supply_iter[railcar,:,:]) <= 0:
                            if move_flag == 0:
                                prev_direct = platoon.direct
                                if platoon.len_platoon() > n_stack:
                                    platoon.shift_platoon()
                                    travel_flag = 1
                                if prev_direct != platoon.direct:
                                    direct_flag = 1;
                        else:
                            move_flag = 1

                    round_flag += 1;
                    # include travel flag because the platoon moves
                    if sum(rail_truck_array) == 0:
                        # if no contianers were transferred from the current crop of contianers break away immediately
                        break;
                    else:
                        trans_flag = 1;
                        if round_flag <= 1:
                            platoon.update_timestamp(max(Entry_time[truck],platoon.timestamp)+2*c*(sum(stor_transfer) > 0)+c*trans_flag)
                        else:
                            platoon.update_timestamp(platoon.timestamp+c*trans_flag)
                        print(f"The updated timestamp is {platoon.timestamp}")
                        OT_Leave[truck].append(platoon.timestamp+c*trans_flag)
                        #OT_Leave[truck].append(max(Entry_time[truck]+2*c*(sum(stor_transfer) > 0)+2*c*trans_flag*round_flag,platoon.timestamp))
                        print(f"Time of direct product transfer is {OT_Leave[truck][-1]}")

                    if travel_flag == 0:
                        break;
                    else:
                        print(f"The platoon at the dock moves")

            if platoon.remain(Supply_iter) == 0:
                #print(f"Important: Platoon emptied out here")
                if prev_platoon_dep == 0:
                    prev_platoon_dep = platoon.timestamp
                    Leave_platoon[platoon_ind] = prev_platoon_dep
                        
                        
                   # if the railcar didn't move do not engage the truck with the same set of railcars.
                   
                    # if platoon.remain(Supply_iter) == 0:
                    #     prev_platoon_dep = platoon.timestamp
                    

            if sum(Demand_iter[truck,:]) <= 0: 
                sel_door = Inv_Stack_Door[truck]
                # changeover_time is the time taken to dock between trucks
                Leave_time[truck] = max(OT_Leave[truck])
                print(f"The truck {truck} leaves at {Leave_time[truck]}")
                Earl_Avail_Stack[sel_door-1] = Leave_time[truck]+changeover_time
                # idea: get the next truck and then accordingly dock, also try to insert pre-empted trucks by first completing a exploratory search
                if cnt < len(Seq_OT)-1:
                    #print(f"The cnt is {cnt}")
                    cnt += 1
                    next_sel_truck = Seq_OT[cnt]
                    print(f"The new truck to arrive is {next_sel_truck}")
                    Inv_Stack_Door[next_sel_truck] = sel_door
                    req_time = max(Arrival_trucks[next_sel_truck],Earl_Avail_Stack[sel_door-1])
                    Entry_time[next_sel_truck] = req_time
                    OT_dock = insert_truck(OT_dock,truck,next_sel_truck,req_time,Entry_time)
                    print(f"The inserted OT_dock is {OT_dock}")

            else:
                index += 1

            
            # if sum(np.array([sum(sum(Supply_iter[railcar,:,:])) for railcar in platoon.list])) <= 0:
            #     platoon_flag = 1;
            #     break;
            # update timestamp only when the railcar moves or products move from cars to trucks
            if index == len(OT_dock):
                # when index moves from the len(OT_dock) to the next, proceed to come out of the loop
                continue;
            else:
                next_truck = OT_dock[index]

                # if Entry_time[next_truck] <= max(Entry_time[truck]+2*c*(sum(stor_transfer) > 0) + c*trans_flag, platoon.timestamp)+wait_time:
                #     # this is the reason for timestamp updating despite the platoon leaving
                #     platoon.update_timestamp(max(Entry_time[truck]++2*c*(sum(stor_transfer) > 0) + c*trans_flag, platoon.timestamp))
                # complex expression below occurs after transferring into the truck or the timestamp of the current set of cars
                if platoon.remain(Supply_iter) > 0:
                    print(f"At the beginning of the third loop the entry of the next truck {next_truck} is {Entry_time[next_truck]}")
                    # need not require updating when the previous truck only takes from storage
                    place_req = 0;
                    if len(OT_Leave[truck]) > 0:
                        place_req = OT_Leave[truck][-1]
                    if trans_flag == 1:
                        platoon.update_timestamp(max(place_req, platoon.timestamp))
                    while Entry_time[next_truck] > max(place_req, platoon.timestamp) + wait_time:
                        print("Check at while loop 3")
                        print(f"At timestamp {platoon.timestamp} the trailers are {platoon.dock_queue()} with the containers being {[np.sum(Supply_iter[railcar,:,:]) for railcar in platoon.dock_queue()]}")
                        Supply_iter,break_signal = Zone.bin_packing(platoon,Supply_iter,max(place_req,platoon.timestamp))
                        print(f"After transferring to storage, at timestamp {platoon.timestamp} the trailers are {platoon.dock_queue()} with the containers being {[np.sum(Supply_iter[railcar,:,:]) for railcar in platoon.dock_queue()]}")
                        
                        # platoon.timestamp will include Entry_time[truck] if there was any direct transfer otherwise 
                        #Supply_iter,break_signal = Zone.bin_packing(platoon,Supply_iter,max(Entry_time[truck]+2*c*(sum(stor_transfer) > 0)+c*trans_flag, platoon.timestamp) )
    
                        if platoon.remain(Supply_iter) == 0:
                            #print(f"Important: Platoon emptied out here")
                            if prev_platoon_dep == 0:
                                prev_platoon_dep = platoon.timestamp
                                Leave_platoon[platoon_ind] = prev_platoon_dep
                            break;
    
                        # if nothing can be transferred break from the loop
                        if break_signal == 0:
                            break;

            #platoon.shift_secondary(Supply_req, max(Entry_time[truck]+2*c*(sum(stor_transfer) > 0) + 2*c*trans_flag, platoon.timestamp) )
            # the below block to catch all instances of if the shift happens before the next truck arrival or not
            if platoon.remain(Supply_iter) == 0:
                #print(f"Important: At the end of the inner loop Platoon emptied out here")
                if prev_platoon_dep == 0:
                    prev_platoon_dep = platoon.timestamp
                    Leave_platoon[platoon_ind] = prev_platoon_dep
            
                    # have to check
            #----------end of the inner OT_dock while loop-------------------------


        # the below block to refresh the entire railroad irrespectibe of wether they are empty or full
        
        if platoon.remain(Supply_iter) == 0:
            #print(f"Important: After the end of the inner loop Platoon emptied out here")
            if prev_platoon_dep == 0:
                prev_platoon_dep = platoon.timestamp
                Leave_platoon[platoon_ind] = prev_platoon_dep
        else:
            # now check if bin packing can take place in the current crop and then move them wholesale
            if platoon.len_platoon() > n_stack:
                print(f"There should be wholesale transfer of rail cars")
                print(f"At timestamp {platoon.timestamp} the trailers are {platoon.dock_queue()} with the containers being {[np.sum(Supply_iter[railcar,:,:]) for railcar in platoon.dock_queue()]}")
                Supply_iter = Zone.second_bin_packing(platoon,Supply_iter,platoon.timestamp)
                print(f"After wholesale moving, at timestamp {platoon.timestamp} the trailers are {platoon.dock_queue()} with the containers being {[np.sum(Supply_iter[railcar,:,:]) for railcar in platoon.dock_queue()]}")
            if platoon.remain(Supply_iter) == 0:
                #print(f"Important: After wholesale removal Platoon emptied out here {prev_platoon_dep}")
                if prev_platoon_dep == 0:
                    prev_platoon_dep = platoon.timestamp
                    Leave_platoon[platoon_ind] = prev_platoon_dep
                # if zone_packing happens it is better to transfer the goods immediately to other remaining trucks
                # therefore immediately implement the function storage to truck

        platoon_stor = platoon.remain(Supply_iter)
        if platoon_stor != 0:
            if prev_platoon_stor == platoon_stor:
                print(f"The deadlocked schedule occurs at platoon number {platoon_list.index(platoon)} ")

                prev_platoon_dep = platoon.timestamp
                Leave_platoon[platoon_ind] = prev_platoon_dep
                break;
        else:
            
            if prev_platoon_dep == 0:
                prev_platoon_dep = platoon.timestamp
                Leave_platoon[platoon_ind] = prev_platoon_dep

        # Iter += 1;
        # if Iter > 4:
        #     break;
        
                
                # sel_truck = Deadline_trucks.index(max([Deadlines_trucks[i] for i in OT_dock]))
                # Earl_Avail_Stack[Inv_Stack_Door[sel_truck]-1] = max([max(OT_Leave[i]) for i in OT_dock])+Change_time;
                
        # truck pre-emption has to be done here, to break the while loop of Supply on railcars

        


-------------------------------------------For platoon number 0 whose arrival is 202.0---------------------------------------------------------------
----------Index number 0 for truck 12 whose arrival is 204.0--------------------
The required set of trucks [12, 20, 21, 11]
Check at while loop 2
The updated timestamp is 219.0
Time of direct product transfer is 234.0
At the beginning of the third loop the entry of the next truck 20 is 318.0
Check at while loop 3
At timestamp 234.0 the trailers are [0, 1, 2, 3] with the containers being [np.int64(3), np.int64(3), np.int64(4), np.int64(2)]
After transferring to storage, at timestamp 249.0 the trailers are [4, 3, 2, 1] with the containers being [np.int64(4), np.int64(0), np.int64(0), np.int64(0)]
Check at while loop 3
At timestamp 249.0 the trailers are [4, 3, 2, 1] with the containers being [np.int64(4), np.int64(0), np.int64(0), np.int64(0)]
After transferring to storage, at timestamp 264.0 the trailers are [4, 3, 2, 1] with the containe

In [10]:
platoon_list[2].history

[np.float64(444.0),
 np.float64(607.0),
 np.float64(622.0),
 np.float64(637.0),
 np.float64(652.0),
 np.float64(667.0),
 np.float64(682.0),
 np.float64(697.0),
 np.float64(712.0),
 np.float64(727.0),
 np.float64(772.0),
 np.float64(787.0),
 np.float64(832.0),
 np.float64(847.0),
 np.float64(892.0),
 np.float64(907.0),
 np.float64(922.0),
 np.float64(937.0),
 np.float64(952.0),
 np.float64(967.0),
 np.float64(982.0),
 np.float64(982.0),
 np.float64(982.0)]

In [11]:
Zone.space

array([0.8, 0.8, 4.4, 4.4])

In [12]:
prev_platoon_dep

np.float64(982.0)

In [13]:
Leave_time

array([  0.,   0.,   0.,   0., 787.,   0.,   0., 697.,   0.,   0.,   0.,
       592., 309.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 637., 667.,
         0.,   0.,   0.,   0., 727., 847.,   0.,   0.,   0.,   0.,   0.,
       472.])

In [14]:
platoon_list[2].remain(Supply_iter)

np.int64(3)

In [15]:
Zone.history

[np.float64(264.0),
 np.float64(279.0),
 np.float64(457.0),
 np.float64(577.0),
 np.float64(937.0)]

In [16]:
Entry_platoon

{0: np.float64(202.0), 1: np.float64(367.0), 2: np.float64(607.0)}

In [17]:
Leave_platoon

{0: np.float64(264.0), 1: np.float64(592.0), 2: np.float64(982.0)}